In [1]:
from finn.util.visualization import showSrc, showInNetron
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper

# Load Model, Clean and Save

In [2]:
model_file = './MY_MBLNET_V2_RESNET_classifier__best_mean_F1__BIPOLAR_Out__QONNX.onnx'

In [3]:
qonnx_clean_filename = '00_clean_model.onnx'
qonnx_cleanup(model_file, out_file=qonnx_clean_filename)

In [4]:
model = ModelWrapper(qonnx_clean_filename)

In [5]:
showInNetron(qonnx_clean_filename)

Serving '00_clean_model.onnx' at http://0.0.0.0:8083


In [6]:
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

In [7]:
model = ModelWrapper(qonnx_clean_filename)

In [8]:
model = model.transform(InferShapes())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())

In [9]:
qonnx_tidy = '01_tidy.onnx'
model.save(qonnx_tidy)

In [10]:
showInNetron(qonnx_tidy)

Stopping http://0.0.0.0:8083
Serving '01_tidy.onnx' at http://0.0.0.0:8083


# Modify Weights

https://github.com/fastmachinelearning/qonnx/blob/main/src/qonnx/core/modelwrapper.py#L150

Names must be retrieved and used, as initializers changes position when graph is modified.

For example, if BatchNorm_0_param0 is modified, it is appended to the end, due to following instruction:
```python
# create and insert new initializer
graph.initializer.append(tensor_init_proto)
```

Therefore, as the model was modified, in the first list of [init_0, init_1, init_2, ...], init_0 is moved, init_1 is jumped and init_2 is the next processed.

In [11]:
import numpy as np

In [12]:
# a = np.array([0.1, -0.1, 0, 2, -2])
# a

In [13]:
# a_abs = np.abs(a)
# idx = (a_abs > 0) * (a_abs < 0.2)
# idx

### Inspect One Quant Initializer with Scale < 1e-10

Find the index of scales < 1e-10 and check if all weights corresponding to that position are zero.

Therefore, as dequantizing implies
$$
x_{dequant} = \frac{x_{quant}}{scale}
$$
scales cannot be zeroed, as division by zero would happen. Those scales can be converted to any value, as weights will remain zero and will not hurt the model.

In [14]:
quant0_0_init = model.get_initializer("Quant_0_param0")
quant0_1_init = model.get_initializer("Quant_0_param1")

print(quant0_0_init.shape)
print(quant0_1_init.shape)

(32, 3, 3, 3)
(32, 1, 1, 1)


In [15]:
q0_1_idx = quant0_1_init < 1e-10
# print(q0_1_idx.nonzero())
zero_idx = np.where(quant0_1_init < 1e-10)
print(f'Indices whith scale < 1e-10:\n{zero_idx}')

print(f'Weights corresponding to those indices:\n{quant0_0_init[zero_idx[0]]}')

Indices whith scale < 1e-10:
(array([17]), array([0]), array([0]), array([0]))
Weights corresponding to those indices:
[[[[-0. -0. -0.]
   [-0. -0. -0.]
   [-0. -0. -0.]]

  [[-0. -0. -0.]
   [-0. -0. -0.]
   [-0. -0. -0.]]

  [[-0. -0.  0.]
   [ 0. -0.  0.]
   [-0. -0. -0.]]]]


### Manually Dequantizing

In [16]:
deq = quant0_0_init / quant0_1_init
print(f'First elements dequantized: {deq[0]}')
print(f'Zero elements dequantized: {deq[zero_idx[0]]}')

First elements dequantized: [[[-1. -3.  2.]
  [-6. -1.  7.]
  [-6.  3.  5.]]

 [[ 3.  1.  4.]
  [-3. -1.  7.]
  [-7. -3. -1.]]

 [[ 4.  2.  4.]
  [-1. -1.  6.]
  [-7. -6. -1.]]]
Zero elements dequantized: [[[[-0. -0. -0.]
   [-0. -0. -0.]
   [-0. -0. -0.]]

  [[-0. -0. -0.]
   [-0. -0. -0.]
   [-0. -0. -0.]]

  [[-0. -0.  0.]
   [ 0. -0.  0.]
   [-0. -0. -0.]]]]


## Modify Initializers

If the value is a Scale in a Convolution Layer, it cannot be converted to zero -> convert to any value, as 0.1, for instance.
Weights of Conv Layer:
- Param 0: weights
- Param 1: scale
- Param 2: zero point
- Param 4: number of bits

Maybe, it is the same with running variance in BatchNorm Layers. It depends on the way it is folded by FINN. Investigate it afterwards.
Values in BN Layer:
- Param 0: gamma
- Param 1: beta
- Param 2: mean
- Param 3: variance

In [17]:
all_inits_names = [init.name for init in model.graph.initializer]

print(f'Number of initializers = {len(all_inits_names)}')

Number of initializers = 376


In [18]:
eps = 1e-10
zero_weight_scale = 1e-1
zero_val = 0

layers_changed = []

for idx, init_name in enumerate(all_inits_names):
    # print(f'Index {idx}:\n\t{init}\n')  
    np_init = model.get_initializer(init_name)
    # print(np_init)
    # print(np_init.flags)

    new_np_init = np.copy(np_init)
    # print(new_np_init.flags)

    np_abs_val = np.abs(new_np_init)
    zero_idx = (np_abs_val < eps) * (np_abs_val > 0)
    if np.all(zero_idx == False):
        print(f'Index = {idx}. {init_name} was not changed, as there were no values under epsilon')
    else:
        layers_changed.append(init_name)
        print(f'Index = {idx}. {init_name} changed, as there were values under epsilon')
        if "Quant" in init_name and "param1" in init_name:
        # It is a scale value, change it accordingly
            zero_now = zero_weight_scale                   
        else:
            zero_now = zero_val     
        new_np_init[zero_idx] = zero_now
        # print(new_np_init)
        model.set_initializer(
            tensor_name = init_name, 
            tensor_value = new_np_init)

Index = 0. BatchNormalization_0_param0 changed, as there were values under epsilon
Index = 1. BatchNormalization_0_param1 changed, as there were values under epsilon
Index = 2. BatchNormalization_0_param2 changed, as there were values under epsilon
Index = 3. BatchNormalization_0_param3 changed, as there were values under epsilon
Index = 4. BatchNormalization_1_param0 changed, as there were values under epsilon
Index = 5. BatchNormalization_1_param1 changed, as there were values under epsilon
Index = 6. BatchNormalization_1_param2 changed, as there were values under epsilon
Index = 7. BatchNormalization_1_param3 changed, as there were values under epsilon
Index = 8. BatchNormalization_2_param0 was not changed, as there were no values under epsilon
Index = 9. BatchNormalization_2_param1 was not changed, as there were no values under epsilon
Index = 10. BatchNormalization_2_param2 was not changed, as there were no values under epsilon
Index = 11. BatchNormalization_2_param3 was not chang

In [19]:
qonnx_min_zeros = 'MobileNetV2_Mini_Resnet__manual_zero_1e-5__best_F1__Bipolar.onnx'
model.save(qonnx_min_zeros)

In [20]:
showInNetron(qonnx_min_zeros)

Stopping http://0.0.0.0:8083
Serving 'MobileNetV2_Mini_Resnet__manual_zero_1e-5__best_F1__Bipolar.onnx' at http://0.0.0.0:8083
